In [1]:
# Import necessary libraries
import pandas as pd
from pycaret.classification import *
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from pycaret.classification import *

In [2]:
# Load and examine the dataset structure
data_train = pd.read_csv('dataset/titanic/train.csv')
display(data_train.info())
display(data_train.describe())
display(data_train.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


None

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# Drop unnecessary columns
data_train.drop(['Name', 'Ticket', 'Embarked', 'Cabin', 'PassengerId'], axis=1, inplace=True)

# Split the data into training and testing sets
train_data, test_data = train_test_split(data_train, random_state=42, train_size=0.8)

In [4]:
# Initialize the setup
clf1 = setup(data = data_train, 
             target = 'Survived', 
             categorical_features = ['Pclass','Sex','Age','Fare'])

,Description,Value
0,Session id,2184
1,Target,Survived
2,Target type,Binary
3,Original data shape,"(891, 7)"
4,Transformed data shape,"(891, 9)"
5,Transformed train set shape,"(623, 9)"
6,Transformed test set shape,"(268, 9)"
7,Numeric features,2
8,Categorical features,4
9,Rows with missing values,19.9%


In [5]:
# Compare models
compare_models(n_select=16)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.7896,0.8340,0.6611,0.7594,0.7033,0.5426,0.5482,0.3380
ridge,Ridge Classifier,0.7833,0.8226,0.6736,0.7426,0.7028,0.5337,0.5381,0.0160
knn,K Neighbors Classifier,0.7623,0.7661,0.6355,0.7095,0.6646,0.4836,0.4891,0.0220
lda,Linear Discriminant Analysis,0.7545,0.8033,0.6525,0.6929,0.6693,0.4750,0.4779,0.0150
qda,Quadratic Discriminant Analysis,0.7529,0.8151,0.7489,0.6599,0.6980,0.4911,0.4985,0.0160
et,Extra Trees Classifier,0.7254,0.7573,0.6147,0.6535,0.6304,0.4126,0.4158,0.0380
gbc,Gradient Boosting Classifier,0.7223,0.7132,0.5942,0.6584,0.6226,0.4040,0.4071,0.0290
rf,Random Forest Classifier,0.7111,0.7602,0.6111,0.6302,0.6178,0.3862,0.3885,0.0440
ada,Ada Boost Classifier,0.7094,0.7092,0.5732,0.6473,0.6035,0.3759,0.3812,0.0260
svm,SVM - Linear Kernel,0.7030,0.7691,0.6246,0.6394,0.6003,0.3741,0.3935,0.0150


[LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=1000,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=2184, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False),
 RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                 max_iter=None, positive=False, random_state=2184, solver='auto',
                 tol=0.0001),
 KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                      metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
                      weights='uniform'),
 LinearDiscriminantAnalysis(covariance_estimator=None, n_components=None,
                            priors=None, shrinkage=None, solver='svd',
                            store_covariance=False, tol=0.0001),
 QuadraticDiscriminantAnalysis(priors=None, reg_param=0.0,
                 

In [6]:
# 創建模型
model = create_model('rf')  # 以隨機森林為例

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8095,0.8707,0.7500,0.7500,0.7500,0.5962,0.5962
1,0.6508,0.7260,0.4583,0.5500,0.5000,0.2351,0.2374
2,0.6667,0.7591,0.6667,0.5517,0.6038,0.3205,0.3248
3,0.6290,0.6190,0.5417,0.5200,0.5306,0.2242,0.2243
4,0.6774,0.6562,0.5417,0.5909,0.5652,0.3096,0.3103
5,0.7097,0.7346,0.5833,0.6364,0.6087,0.3786,0.3795
6,0.7258,0.7807,0.7500,0.6207,0.6792,0.4435,0.4496
7,0.7419,0.8575,0.6250,0.6818,0.6522,0.4477,0.4487
8,0.7097,0.8344,0.5417,0.6500,0.5909,0.3688,0.3725


In [7]:
# 導入 Optuna
import optuna

In [12]:
# 定義 Optuna 的目標函數
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2'])
    }
    model = create_model('rf', **params)
    tuned_model = tune_model(model)
    # 使用 cross_val_score 評估模型並返回平均準確性
    score = pull().iloc[0]['Accuracy']
    return score

In [13]:
# 創建一個研究並優化目標函數
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

[I 2024-12-08 22:20:33,443] A new study created in memory with name: no-name-16aac928-32e5-41fb-8cc6-e88720f3764e


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7937,0.9060,0.6667,0.7619,0.7111,0.5517,0.5547
1,0.6984,0.7842,0.5000,0.6316,0.5581,0.3339,0.3391
2,0.6984,0.7788,0.7083,0.5862,0.6415,0.3852,0.3904
3,0.5968,0.6650,0.3750,0.4737,0.4186,0.1163,0.1182
4,0.6613,0.6645,0.5833,0.5600,0.5714,0.2916,0.2918
5,0.7419,0.7215,0.5833,0.7000,0.6364,0.4389,0.4433
6,0.7581,0.8125,0.7083,0.6800,0.6939,0.4940,0.4943
7,0.7903,0.8947,0.6250,0.7895,0.6977,0.5405,0.5491
8,0.7581,0.8268,0.5833,0.7368,0.6512,0.4698,0.4773


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8413,0.9071,0.7500,0.8182,0.7826,0.6580,0.6595
1,0.6825,0.7703,0.5000,0.6000,0.5455,0.3046,0.3076
2,0.7143,0.7917,0.6667,0.6154,0.6400,0.4038,0.4047
3,0.6129,0.6552,0.4167,0.5000,0.4545,0.1584,0.1600
4,0.6774,0.6749,0.5833,0.5833,0.5833,0.3202,0.3202
5,0.7742,0.7171,0.6250,0.7500,0.6818,0.5090,0.5141
6,0.7581,0.8289,0.7083,0.6800,0.6939,0.4940,0.4943
7,0.8387,0.8969,0.7083,0.8500,0.7727,0.6493,0.6558
8,0.7419,0.8596,0.5833,0.7000,0.6364,0.4389,0.4433


Fitting 10 folds for each of 10 candidates, totalling 100 fits


[I 2024-12-08 22:20:41,341] Trial 0 finished with value: 0.8413 and parameters: {'n_estimators': 395, 'max_depth': 4, 'min_samples_split': 11, 'min_samples_leaf': 13, 'max_features': 'log2'}. Best is trial 0 with value: 0.8413.


...

[I 2024-12-08 22:27:43,156] Trial 48 finished with value: 0.8413 and parameters: {'n_estimators': 888, 'max_depth': 9, 'min_samples_split': 6, 'min_samples_leaf': 12, 'max_features': 'log2'}. Best is trial 0 with value: 0.8413.


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7937,0.9006,0.6667,0.7619,0.7111,0.5517,0.5547
1,0.6984,0.7724,0.5000,0.6316,0.5581,0.3339,0.3391
2,0.6984,0.7703,0.7083,0.5862,0.6415,0.3852,0.3904
3,0.5968,0.6541,0.3750,0.4737,0.4186,0.1163,0.1182
4,0.6774,0.6732,0.6250,0.5769,0.6000,0.3305,0.3312
5,0.7419,0.7105,0.5417,0.7222,0.6190,0.4299,0.4401
6,0.7581,0.8114,0.7083,0.6800,0.6939,0.4940,0.4943
7,0.7903,0.8750,0.6250,0.7895,0.6977,0.5405,0.5491
8,0.7581,0.8169,0.5833,0.7368,0.6512,0.4698,0.4773


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8413,0.9071,0.7500,0.8182,0.7826,0.6580,0.6595
1,0.6825,0.7703,0.5000,0.6000,0.5455,0.3046,0.3076
2,0.7143,0.7917,0.6667,0.6154,0.6400,0.4038,0.4047
3,0.6129,0.6552,0.4167,0.5000,0.4545,0.1584,0.1600
4,0.6774,0.6749,0.5833,0.5833,0.5833,0.3202,0.3202
5,0.7742,0.7171,0.6250,0.7500,0.6818,0.5090,0.5141
6,0.7581,0.8289,0.7083,0.6800,0.6939,0.4940,0.4943
7,0.8387,0.8969,0.7083,0.8500,0.7727,0.6493,0.6558
8,0.7419,0.8596,0.5833,0.7000,0.6364,0.4389,0.4433


Fitting 10 folds for each of 10 candidates, totalling 100 fits


[I 2024-12-08 22:27:52,283] Trial 49 finished with value: 0.8413 and parameters: {'n_estimators': 441, 'max_depth': 20, 'min_samples_split': 10, 'min_samples_leaf': 14, 'max_features': 'log2'}. Best is trial 0 with value: 0.8413.


In [14]:
# 獲取最佳試驗
best_trial = study.best_trial
print(f'Best trial: {best_trial.params}')

Best trial: {'n_estimators': 395, 'max_depth': 4, 'min_samples_split': 11, 'min_samples_leaf': 13, 'max_features': 'log2'}


In [15]:
# 使用最佳參數創建並調優模型
best_params = best_trial.params
model = create_model('rf', **best_params)
tuned_model = tune_model(model)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7937,0.9060,0.6667,0.7619,0.7111,0.5517,0.5547
1,0.6984,0.7842,0.5000,0.6316,0.5581,0.3339,0.3391
2,0.6984,0.7788,0.7083,0.5862,0.6415,0.3852,0.3904
3,0.5968,0.6650,0.3750,0.4737,0.4186,0.1163,0.1182
4,0.6613,0.6645,0.5833,0.5600,0.5714,0.2916,0.2918
5,0.7419,0.7215,0.5833,0.7000,0.6364,0.4389,0.4433
6,0.7581,0.8125,0.7083,0.6800,0.6939,0.4940,0.4943
7,0.7903,0.8947,0.6250,0.7895,0.6977,0.5405,0.5491
8,0.7581,0.8268,0.5833,0.7368,0.6512,0.4698,0.4773


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8413,0.9071,0.7500,0.8182,0.7826,0.6580,0.6595
1,0.6825,0.7703,0.5000,0.6000,0.5455,0.3046,0.3076
2,0.7143,0.7917,0.6667,0.6154,0.6400,0.4038,0.4047
3,0.6129,0.6552,0.4167,0.5000,0.4545,0.1584,0.1600
4,0.6774,0.6749,0.5833,0.5833,0.5833,0.3202,0.3202
5,0.7742,0.7171,0.6250,0.7500,0.6818,0.5090,0.5141
6,0.7581,0.8289,0.7083,0.6800,0.6939,0.4940,0.4943
7,0.8387,0.8969,0.7083,0.8500,0.7727,0.6493,0.6558
8,0.7419,0.8596,0.5833,0.7000,0.6364,0.4389,0.4433


Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [17]:
# 評估調優後的模型
evaluate_model(tuned_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [18]:
# 保存調優後的模型
save_model(tuned_model, 'best_tuned_model_optuna')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None, include=['SibSp', 'Parch'],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                               fill_value=None,
                                                               keep_empty_features=False,
                                                               missing_values=nan,
                                                               strategy='mean'))),
                 ('categorical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['Pclass', 'Sex', 'Age', 'Fare...
                  RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                         class_weight={}, criterion='entropy',
                          